In [39]:
import os
import logging
import numpy as np
import pandas as pd

import riskflow_jupyter

In [40]:
import importlib
importlib.reload(riskflow_jupyter)

<module 'riskflow_jupyter' from '/home/vretiel/PycharmProjects/riskflow/riskflow_jupyter.py'>

In [41]:
rundate = '2022-07-07'
if os.name=='nt':
    path = os.path.join('U:\\CVA_JSON', rundate)
    path_transform={
        '\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData':
        '\\\\ICMJHBMVDROPUAT\\AdaptiveAnalytics\\Inbound\\MarketData'}
else:
    path = os.path.join('/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON', rundate)
    path_transform={
            '//ICMJHBMVDROPPRD/AdaptiveAnalytics/Inbound/MarketData': 
            '/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON'}
    
w=riskflow_jupyter.Workbench(
    path=path, 
    path_transform=path_transform,
    file_transform={
    'CVAMarketData_Calibrated.dat': 'CVAMarketData_Calibrated_New.json',
    'MarketData.dat': 'MarketData.json'})



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/vretiel/miniconda3/lib/python3.8/site-packages/ipywidgets/widgets/widget.py", line 217, in m
    return(method(self, *args, **kwargs))
  File "/home/vretiel/miniconda3/lib/python3.8/site-packages/ipywidgets/widgets/widget.py", line 676, in _handle_msg
    self.set_state(state)
  File "/home/vretiel/miniconda3/lib/python3.8/site-packages/ipywidgets/widgets/widget.py", line 545, in set_state
    self.set_trait(name, from_json(sync_data[name], self))
  File "/home/vretiel/miniconda3/lib/python3.8/contextlib.py", line 120, in __exit__
    next(self.gen)
  File "/home/vretiel/miniconda3/lib/python3.8/site-packages/traitlets/traitlets.py", line 1216, in hold_trait_notifications
    self.notify_change(change)
  File "/home/vretiel/miniconda3/lib/python3.8/site-packages/ipywidgets/widgets/widget.py", line 606, in notify_change
    super(Widget, self).notify_change(change)
  File "/home/vretiel/miniconda3/lib/python3.8/site-packages/traitlets/tra

In [42]:
w.context.current_cfg.deals['Calculation']

{'Object': 'CreditMonteCarlo',
 'Base_Date': Timestamp('2022-07-07 00:00:00'),
 'Collateral_Algorithm': 'Do Nothing Collateral Algorithm',
 'Currency': 'ZAR',
 'Adjust_To_External_MtM': 'No',
 'Base_Time_Grid': <riskflow.utils.Offsets at 0x7f7898623880>,
 'Sub_Calculations': [{'Name': 'Bank',
   'Filter': '',
   'Tags': '',
   'Aggregation_Mode': 'Auto',
   'Exposure_View': 'Bank',
   'Settlement_Adjustment': 0,
   'Liquidation_Adjustment': 0},
  {'Name': 'Counterparty',
   'Filter': '',
   'Tags': '',
   'Aggregation_Mode': 'Auto',
   'Exposure_View': 'Counterparty',
   'Settlement_Adjustment': 0,
   'Liquidation_Adjustment': 0}],
 'Scenarios': 100,
 'Antithetic': 'No',
 'Sampling': 'Unstratified',
 'Random_Seed': '1',
 'Output_Detail': 'Quantiles',
 'Use_Dynamic_Dates': 'Yes',
 'Scenario_Time_Grid': <riskflow.utils.Offsets at 0x7f7898623370>,
 'Max_Horizon': '',
 'Deflation_Interest_Rate': 'ZAR-SWAP',
 'Deterministic_Scenarios': {'Deterministic_Scenarios': 'Off',
  'Scenario_Path': '

In [3]:
w.context.current_cfg.deals['Deals']['Children'][0]['Children'][0]['Instrument'].field

IndexError: list index out of range

In [ ]:
rundate = '2022-07-07'
if os.name=='nt':
    path = os.path.join('U:\\CVA_JSON', rundate)
    path_transform={
        '\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData':
        '\\\\ICMJHBMVDROPUAT\\AdaptiveAnalytics\\Inbound\\MarketData'}
else:
    path = os.path.join('/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON', rundate)
    path_transform={
            '//ICMJHBMVDROPPRD/AdaptiveAnalytics/Inbound/MarketData': 
            '/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON'}
    
cx = riskflow_jupyter.rf.Context(
    path_transform=path_transform,
    file_transform={
        'CVAMarketData_Calibrated.dat': 'CVAMarketData_Calibrated_New.json',
        'MarketData.dat': 'MarketData.json'
    })

In [ ]:
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_Russellstone_Treasury_ISDA.json'))
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_BNP_Paribas__Paris__ISDA.json'))
cx.load_json(os.path.join(path, 'InputAAJ_CrB_Citibank_NA_NY_ISDA.json'))
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_Natixis_ISDA.json'))



In [ ]:
calc, output = cx.run_job()